In [1]:

import pandas as pd

path = "/root/Desktop/cv-corpus-9.0-2022-04-27/en/train.tsv"
df = pd.read_csv(path, sep='\t')
df.head(3)


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment
0,35722a754ae8192dcbcd7ba3640efab6dff0c3d13ef03e...,common_voice_en_28322365.mp3,"Feeling threatened by these actions, the Moros...",2,0,NaN,NaN,NaN,en,NaN
1,35722a754ae8192dcbcd7ba3640efab6dff0c3d13ef03e...,common_voice_en_28322367.mp3,"King, Big Joe Williams, and Ace Atkins.",2,0,NaN,NaN,NaN,en,NaN
2,35722a754ae8192dcbcd7ba3640efab6dff0c3d13ef03e...,common_voice_en_28322368.mp3,And this is where we find the roots of the ear...,2,0,NaN,NaN,NaN,en,NaN


#### Load paths and wavs:

In [2]:

import os

wavs = "/root/Desktop/wavs"
converted = [i for i in os.listdir(wavs)]
samples = df["path"].values
samples = samples[: ]
sample2text = {k: v for k, v in zip(df["path"].values, df["sentence"].values)} 



#### Text and path:

In [3]:

import re
from multiprocessing import Pool, cpu_count


def text_filter(text):
    text = text.lower()
    text = re.sub("[^\w ']", " ", text)
    text = re.sub(" +", " ", text)
    return text.strip()


def function(n):
    path = os.path.join(wavs, n.replace('mp3', 'wav'))
    if os.path.exists(path):
        return path, text_filter(sample2text[n]) 

    
print(f'Starting on {cpu_count()} cores')

with Pool() as pool:
    res = pool.map(function, samples)

res = [i for i in res if i]
print("Found: ", len(res))



Starting on 16 cores
Found:  397792


#### Play random sample:

In [4]:

import os
from random import choice
from pydub import AudioSegment
from pydub.playback import play


sample = choice(res)
audio_segment = AudioSegment.from_wav(os.path.join(wavs, sample[0]))
print(f"Play sample: {sample[1]}")
play(audio_segment)


Play sample: myrdal wanted to combine and integrate the two extremes


Input #0, wav, from '/tmp/tmpmyysacjz.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:05.66, bitrate: 256 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 16000 Hz, 1 channels, s16, 256 kb/s


#### Writing the output:

In [9]:
import re
import csv

# wav_filename,wav_filesize,transcript

def make_csv(path, data):
    with open(path, 'w', newline='') as csvfile:
        fieldnames = ['wav_filename' ,'wav_filesize' , 'transcript']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for c, (x, y) in enumerate(data):
            writer.writerow({
                'wav_filename': x,
                'wav_filesize': os.stat(x).st_size,
                'transcript': y,
            })
            os.system(f"cp {x} /root/Desktop/test")
            if c == 20:
                break
            
        
train = res[: - int(len(res) * 0.2)]
test = res[- int(len(res) * 0.2): ]

make_csv("/root/Desktop/train.csv", train)
make_csv("/root/Desktop/test.csv", test)